In [ ]:
import pandas as pd

index_data = pd.read_csv('sh000001.csv', parse_dates=['date'])
index_data = index_data[['date', 'high', 'low', 'close', 'change']]
index_data.sort('date', inplace=True)
N1 = 20
N2 = 10
index_data['n1极大值'] =  pd.rolling_max(index_data['high'], N1)
index_data['n1极大值'].fillna(value=pd.expanding_max(index_data['high']), inplace=True)
index_data['n2极小值'] =  pd.rolling_min(index_data['low'], N1)
index_data['n2极小值'].fillna(value=pd.expanding_min(index_data['low']), inplace=True)
buy_index = index_data[index_data['close'] > index_data['n1极大值'].shift(1)].index
index_data.loc[buy_index, '信号'] = 1
sell_index = index_data[index_data['close'] < index_data['n2极小值'].shift(1)].index
index_data.loc[sell_index, '信号'] = 0
index_data['仓位'] = index_data['信号'].shift(1)
index_data['仓位'].fillna(method='ffill', inplace=True)
index_data = index_data[index_data['date'] >= pd.to_datetime('19930101')]
index_data['资金指数'] = (index_data['change'] * index_data['仓位'] + 1.0).cumprod()
initial_idx = index_data.iloc[0]['close'] / (1 + index_data.iloc[0]['change'])
index_data['资金指数'] *= initial_idx
index_data[['date', 'high', 'low', 'close', 'change', 'n1极大值',
            'n2极小值', '仓位', '资金指数']].to_csv('turtle.csv', index=False, encoding='gbk')
index_data['每日涨跌'] = index_data['change'] * index_data['仓位']
year_rtn = index_data.set_index('date')[['change', '每日涨跌']].\
               resample('A', how=lambda x: (x+1.0).prod() - 1.0) * 100
print(year_rtn)